# Research list:
- Handling outliers in timeseries forecasting

In [1]:

import time

%matplotlib inline

## Importing Libraries
import sys
import numbers
import time
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from functools import reduce

import pmdarima as pmd
import statsmodels.api as sm 
from scipy.stats import normaltest

from darts import TimeSeries
from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    Prophet,
    ExponentialSmoothing,
    ARIMA,
    AutoARIMA,
    Theta, 
    RegressionEnsembleModel)               # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<

from darts.metrics import mape, mase, mae, mse, ope, r2_score, rmse, rmsle
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis
from darts.dataprocessing.transformers.boxcox import BoxCox
from darts.utils.utils import ModelMode           # new 
from darts.utils.missing_values import fill_missing_values

from darts.datasets import ( 
    AirPassengersDataset, AusBeerDataset, GasRateCO2Dataset, HeartRateDataset, 
    IceCreamHeaterDataset, MonthlyMilkDataset, SunspotsDataset)


import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)



TRACE = False                 # print also the suboptimal models while SARIMA tuning process is running
MSEAS = 12                    # seasonality default
ALPHA = 0.1                  # significance level default

# Prepare the data

In [2]:
## load data

mdf = pd.read_csv('./Resources/RK_RFCU_Gross_Chargeoffs.csv?')
mdf['MONTH_END_DT'] = pd.to_datetime(mdf['MONTH_END_DT'])
print(mdf.info())

#Create a DF of only RFCU data
rfcu_df = mdf[['MONTH_END_DT', 'RFCU_CC_Chargeoff']]
rfcu_df = rfcu_df[rfcu_df['RFCU_CC_Chargeoff'] != 0]
# rfcu_df.set_index(keys='MONTH_END_DT', inplace=True, drop=True)

#Create a DF of only RK data
rk_df = mdf[['MONTH_END_DT', 'RFCU_CC_Chargeoff']]
rk_df = rk_df[rk_df['RFCU_CC_Chargeoff'] != 0]
# rk_df.set_index(keys='MONTH_END_DT', inplace=True, drop=True)

# Select which df you want to put into the timeseries
# series = rk_df
series = rfcu_df
mdf = rfcu_df
mdf.columns = ['month', 'chargeoff_amt']
series.columns = ['month', 'chargeoff_amt']
series.set_index(keys='month', inplace=True, drop=True)

FileNotFoundError: [Errno 2] No such file or directory: './Resources/RK_RFCU_Gross_Chargeoffs.csv?'

In [ ]:
# Connect to snowflake
from snowflake.snowpark import Session
import json
credentials = json.load(open('secrets.json'))["SNOWFLAKE_CONNECTION"]

session = Session.builder.configs(credentials).create()

In [ ]:
session.use_database("Covid19")
session.use_schema("public")
mdf = session.table("CDC_testing")

In [ ]:
mdf.to_pandas()

SnowparkFetchDataException: (1406): Failed to fetch a Pandas Dataframe. The error is: 255002: Optional dependency: 'pandas' is not installed, please see the following link for install instructions: https://docs.snowflake.com/en/user-guide/python-connector-pandas.html#installation

In [ ]:
series.describe()

In [ ]:
# plot the observations

plt.figure(100, figsize=(12, 5))
series.plot()

# Check for Outliers considering seasonality and trend

In [ ]:
# Check for outliers despite seasonality and trend
#compute seasonal decomposition

from statsmodels.tsa.seasonal import seasonal_decompose
decomp = seasonal_decompose(series, period=12)

#plot decomp
fig = decomp.plot()
fig.set_size_inches((16, 11))
fig.tight_layout()
plt.show()

In [ ]:
#add decomp series to the df

series['trend'] = decomp.trend
series['seasonal'] = decomp.seasonal
series['resid'] = decomp.resid

In [ ]:
#define 90% upper and lower bounds

z_val = 3
series['resid_UB'] = series['resid'].mean() + z_val * series['resid'].std()
series['resid_LB'] = series['resid'].mean() - z_val * series['resid'].std()

In [ ]:
#add anomaly flags

series['is_anomaly'] = (np.where((series['resid'] > series['resid_UB'])
                             | (series['resid'] < series['resid_LB']),
                             1,0))

In [ ]:
# display anomalies
display(series[series['is_anomaly'] == 1])

In [ ]:
# Drop the anamolies, and impute with darts
series = series[series['is_anomaly'] != 1]

ts_series = series['chargeoff_amt']
ts = TimeSeries.from_series(ts_series, fill_missing_dates=True)
ts = fill_missing_values(ts, fill='auto')

In [ ]:
# Plot the new time series
plt.figure(100, figsize=(12, 5))
ts.plot()

# Plot and check ACF

In [ ]:
# Plot ACF to check for autocorrelation
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
plot_acf(mdf.chargeoff_amt)
result = adfuller(rfcu_df.chargeoff_amt.dropna())
print('p-value: ', result[1])

In [ ]:
#First order differencing
f, axs = plt.subplots(1, 2, figsize=(15, 8))
axs[0] = f.add_subplot(121)
axs[0].set_title('1st Order Differencing')
axs[0].plot(mdf.chargeoff_amt.diff())
#Plot PCAF and check it


plot_acf(mdf.chargeoff_amt.diff().dropna(), ax=axs[1])
plt.show()

#Test statistic
result = adfuller(mdf.chargeoff_amt.diff().dropna())
print('p-value: ', result[1])

In [ ]:
#Second order differencing
f, axs = plt.subplots(1, 2, figsize=(15, 8))
axs[0].set_title('2nd Order Differencing')
axs[0].plot(mdf.chargeoff_amt.diff().diff())


plot_acf(mdf.chargeoff_amt.diff().diff().dropna(), ax=axs[1])
plt.show()
result = adfuller(mdf.chargeoff_amt.diff().diff().dropna())
print('p-value: ', result[1])

# Checking for seasonality. mseas will be used as an input to models

In [ ]:
# check for seasonality, via ACF

for m in range(2, 25):
    is_seasonal, mseas = check_seasonality(ts, m=m, alpha=ALPHA)
    if is_seasonal:
        break
    else:
        mseas = 12

print("seasonal? " + str(is_seasonal))
if is_seasonal:
    print('There is seasonality of order {}.'.format(mseas))
print(f'mseas = {mseas}')

In [ ]:
## split train and test data

# split position: if string, then interpret as Timestamp
# if int, then interpretation as index
# if loat, then interpretation as %split

# if isinstance(TRAIN, numbers.Number):
#     split_at = TRAIN
# else:
#     split_at = pd.Timestamp(TRAIN)
# train, val = ts.split_before(split_at)

from darts.utils import model_selection
# train, val = model_selection.train_test_split(ts, test_size=0.33, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
train, val = model_selection.train_test_split(ts, test_size=0.25, axis=0, vertical_split_type='model-aware', horizon=3, input_size=3, lazy=False)

plt.figure(101, figsize=(12, 5))
train.plot(label='training')
val.plot(label='validation')
plt.legend();

# Functions for pipeline. Building Models and Results

In [3]:
# compute accuracy metrics and processing time

def accuracy_metrics(act, forecast, resid, t_start):
    sr = resid.pd_series()
    sa = act.pd_series()
    n_act = len(act)
    res_mape = mape(act, forecast)
    res_mae = mae(act, forecast)
    res_r2 = r2_score(act, forecast)
    res_rmse = rmse(act, forecast)
    res_rmsle = rmsle(act, forecast)
    res_pe = sr / sa
    res_rmspe = np.sqrt(np.sum(res_pe**2) / n_act)    # root mean square percentage error

    res_time = time.perf_counter() - t_start
    
    res_mean = np.mean(sr)
    res_std = np.std(sr)                               # std error of the model = std deviation of the noise
    res_se = res_std / np.sqrt(n_act)                  # std error in estimating the mean
    res_sefc = np.sqrt(res_std + res_se**2)            # std error of the forecast
    
    res_accuracy = {"MAPE":res_mape,"RMSE":res_rmse, "-R squared":-res_r2, 
        "se": res_sefc, "time":res_time}
    return res_accuracy

In [4]:
# Chance to set mseas manually to check model performance. Comment out to use mseas from above
# mseas = 12

In [5]:
## fit the chosen forecaster model and compute predictions

def eval_model(model):
    t_start =  time.perf_counter()
    strmodel = str(model)[:30]
    print("beginning: " + strmodel)


    # fit the model and compute predictions
    n_val = len(val)
    res = model.fit(train)
    forecast = model.predict(n_val)


    # for naive forecast, concatenate seasonal fc with drift fc
    if model == m_naive:
        if is_seasonal:
            fc_drift = forecast
            modelS = NaiveSeasonal(K=mseas)
            modelS.fit(train)
            fc_seas = modelS.predict(len(val))
            forecast = fc_drift + fc_seas - train.last_value()


    resid = forecast - val
    res_accuracy = accuracy_metrics(val, forecast, resid, t_start)
    
    
    results = [forecast, res_accuracy]
    
    print("completed: " + strmodel + ":" + str(res_accuracy["time"]) + " sec")
    return results

In [6]:
# prepare Naive forecaster

m_naive = NaiveDrift()
print("model:", m_naive)

model: Naive drift model


In [7]:
# prepare Exponential Smoothing forecaster

# SET EXPONENTIAL SMOOTHING MODE:
# exp_mode = 'add'
exp_mode = 'mult'

# Do research here to understand more about using exponential smoothing forecaster when data is seasonal.
if is_seasonal:
    if exp_mode == 'add':
        m_expon = ExponentialSmoothing( trend=ModelMode.ADDITIVE, 
                                    damped=False, 
                                    seasonal=ModelMode.ADDITIVE, 
                                    seasonal_periods=mseas) 
    else:
        m_expon = ExponentialSmoothing( trend=ModelMode.MULTIPLICATIVE, 
                                    damped=False, 
                                    seasonal=ModelMode.MULTIPLICATIVE, 
                                    seasonal_periods=mseas) 
else:
    m_expon = ExponentialSmoothing()

print("model:", m_expon)

NameError: name 'is_seasonal' is not defined

In [8]:
# prepare Prophet forecaster

m_prophet = Prophet()    #frequency=mseas)

# m_prophet = Prophet(add_seasonalities={'name': 'Mthly','seasonal_periods': 12, 'fourier_order': 1})

print("model:", m_prophet)

model: Prophet


In [9]:
# prepare ARIMA forecaster

y = np.asarray(ts.pd_series())
# get order of first differencing: the higher of KPSS and ADF test results
n_kpss = pmd.arima.ndiffs(y, alpha=ALPHA, test='kpss', max_d=4)
n_adf = pmd.arima.ndiffs(y, alpha=ALPHA, test='adf', max_d=4)
n_diff = max(n_adf, n_kpss)
min_diff = min(n_adf, n_kpss)

# get order of seasonal differencing: the higher of OCSB and CH test results
n_ocsb = pmd.arima.OCSBTest(m=mseas).estimate_seasonal_differencing_term(y)
n_ch = pmd.arima.CHTest(m=mseas).estimate_seasonal_differencing_term(y)
ns_diff = max(n_ocsb, n_ch, is_seasonal * 1)

# set up the ARIMA forecaster
m_arima = AutoARIMA(
    start_p=1, d=min_diff, start_q=1,
    max_p=4, max_d=n_diff, max_q=4,
    start_P=0, D=ns_diff, start_Q=0, m=max(4,mseas), seasonal=is_seasonal,
    max_P=3, max_D=1, max_Q=3,
    max_order=5,                       # p+q+p+Q <= max_order
    stationary=False, 
    information_criterion="bic", alpha=ALPHA, 
    test="kpss", seasonal_test="ocsb",
    stepwise=True, 
    suppress_warnings=True, error_action="trace", trace=TRACE, with_intercept="auto")
print("model:", m_arima)

NameError: name 'ts' is not defined

In [10]:
# prepare Theta forecaster

# search space for best theta value: check 100 alternatives
thetas = 2 - np.linspace(-10, 10, 100)

# initialize search
best_mape = float('inf')
best_theta = 0
# search for best theta among 50 values, as measured by MAPE
for theta in thetas:
    model = Theta(theta)
    res = model.fit(train)
    pred_theta = model.predict(len(val))
    res_mape = mape(val, pred_theta)

    if res_mape < best_mape:
        best_mape = res_mape
        best_theta = theta

m_theta = Theta(best_theta)   # best theta model among 100
print("model:", m_theta)

NameError: name 'train' is not defined

# Run the models

In [11]:
# laundry list of forecasters to run
# Comment each individual out to remove if won't run due to data length, etc.

models = [ 
    m_expon, 
    m_theta, 
    m_arima,
    m_naive, 
    m_prophet]



NameError: name 'm_expon' is not defined

In [12]:
# call the forecasters one after the other

model_predictions = [eval_model(model) for model in models]

NameError: name 'models' is not defined

In [13]:
# RUN the forecasters and tabulate their prediction accuracy and processing time

df_acc = pd.DataFrame.from_dict(model_predictions[0][1], orient="index")
df_acc.columns = [str(models[0])]

for i, m in enumerate(models):
    if i > 0: 
        df = pd.DataFrame.from_dict(model_predictions[i][1], orient="index")
        df.columns = [str(m)]
        df_acc = pd.concat([df_acc, df], axis=1)
    i +=1

pd.set_option("display.precision",3)
#df_acc.style.highlight_min(color="lightgreen", axis=1).highlight_max(color="yellow", axis=1)
display(df_acc)

NameError: name 'model_predictions' is not defined

In [14]:
# plot the forecasts

pairs = math.ceil(len(models)/2)                    # how many rows of charts
fig, ax = plt.subplots(pairs, 2, figsize=(20, 5 * pairs))
ax = ax.ravel()

for i,m in enumerate(models):
        ts.plot(label="actual", ax=ax[i])
        model_predictions[i][0].plot(label="prediction: "+str(m), ax=ax[i])
        
        mape_model =  model_predictions[i][1]["MAPE"]
        time_model =  model_predictions[i][1]["time"]
        ax[i].set_title("\n\n" + str(m)[:30] + ": MAPE {:.1f}%".format(mape_model) + " - time {:.1f}sec".format(time_model))

        ax[i].set_xlabel("")
        ax[i].legend()

NameError: name 'models' is not defined

In [15]:
act = val

resL = {}
resN = {} 
for i,m in enumerate(models):
        pred = model_predictions[i][0]
        resid = pred - act
        sr = resid.pd_series() 

        resL[str(m)] = sm.stats.acorr_ljungbox(sr, lags=[5], return_df=False)[1][0]
        resN[str(m)] = normaltest(sr)[1]

        
print("\nLjung-Box test for white-noise residuals: p-value > alpha?")
_ = [print(key,":",value) for key,value in resL.items()]

print("\ntest for normality of residuals: p-value > alpha?")
_ = [print(key,":",value) for key,value in resN.items()]



NameError: name 'val' is not defined

In [16]:
# investigate the residuals in the validation dataset

act = val
df_desc = pd.DataFrame()

for i,m in enumerate(models):
        pred = model_predictions[i][0]
        resid = pred - act
        resid = resid.pd_dataframe()

        df_desc = pd.concat([df_desc, resid.describe()], axis=1)

        #plot_residuals_analysis(resid);
        #plt.title(str(m))
        

NameError: name 'val' is not defined

In [17]:
# descriptive statistics of the forecast series
df_desc.columns = [str(m) for m in models]
print(df_desc)

NameError: name 'models' is not defined

# combine the individual models in an ensemble forecast:

In [18]:
# ensemble forecast evaluation function:
# the eval function calls the methods contained in the list 'models'

def ensemble_eval(train, val, models):
    t_start =  time.perf_counter()
    n_train = 12              # use 50 observation to train the ensemble model
    print(n_train)
    n_val = len(val)            # forecast as many periods as are in the valuation dataset


    # compute predictions
    # ensemble_model = RegressionEnsembleModel(forecasting_models=models, regression_train_n_points=n_train)
    # new Jan 022: RegressionEnsembleModel class no longer accepts as argument the list "models" of previously instantiated models
    # instead, explicitly list each of the forecast methods
    ensemble_model = RegressionEnsembleModel(
                                forecasting_models=[    Theta(best_theta), 
                                                        Prophet(), 
                                                        AutoARIMA(), 
                                                        NaiveDrift(),
                                                        ExponentialSmoothing(
                                                                            trend=ModelMode.MULTIPLICATIVE, 
                                                                            damped=False, 
                                                                            seasonal=ModelMode.MULTIPLICATIVE, 
                                                                            seasonal_periods=mseas)]
                                 ,regression_train_n_points=n_train)



    ensemble_model.fit(train)
    forecast = ensemble_model.predict(n_val)
    resid = forecast - val


    res_accuracy = accuracy_metrics(val, forecast, resid, t_start)


    # plot the ensemble forecast
    ts.plot(label="actual")
    forecast.plot(label="Ensemble forecast")
    plt.title("MAPE = {:.2f}%".format(res_accuracy["MAPE"]))
    plt.legend();


    results = [forecast, res_accuracy]
    return results


In [19]:
# call the ensemble eval function with all 5 forecasters:

# column headers and model selection:
col_heads = ["Expon", "Theta", "ARIMA", "Naive", "Prophet", "avg", "Ensemble"]
models2 = models

# run the ensemble forecast
print("Ensemble of all 5 forecasters:")
res_ensemble = ensemble_eval(train, val, models2)


NameError: name 'models' is not defined

In [20]:
# collect the accuracy metrics

df_acc2 = df_acc.copy()
df_acc2["avg"] = df_acc2.mean(axis=1)
df_acc2["Ensemble"] = pd.Series(res_ensemble[1])
df_acc2.columns = col_heads
df_acc2.style.highlight_min(color="lightgreen", axis=1).highlight_max(color="yellow", axis=1)

NameError: name 'df_acc' is not defined

In [21]:
#resid = res_ensemble[2]

resid = res_ensemble[0] - val
sr = resid.pd_series()
#plot_residuals_analysis(resid);
#plt.title("Ensemble forecast")

NameError: name 'res_ensemble' is not defined

In [22]:
resL = sm.stats.acorr_ljungbox(sr, lags=[5], return_df=False)[1][0]
resN = normaltest(sr)[1]
   
print("\nLjung-Box test for white-noise residuals: p-value > alpha?")
print(resL)

print("\ntest for normality of residuals: p-value > alpha?")
print(resN)

NameError: name 'sr' is not defined

In [23]:
# plot the forecast scenario, and now include the ensemble

pairs = math.ceil(len(models)/2)                    # how many rows of charts
fig, ax = plt.subplots(pairs, 2, figsize=(20, 5 * pairs))
ax = ax.ravel()

for i,m in enumerate(models):
        ts.plot(label="actual", ax=ax[i])
        model_predictions[i][0].plot(label="prediction: "+str(m)[:30], ax=ax[i])
        rmse_model =  model_predictions[i][1]["RMSE"]
        time_model =  model_predictions[i][1]["time"]
        ax[i].set_title("\n\n" + str(m)[:30] + ": RMSE {:.0f}".format(rmse_model) + " - time {:.1f}sec".format(time_model))
        ax[i].set_xlabel("")
        ax[i].legend()


# add the ensemble:
ts.plot(label="actual", ax=ax[i+1])
res_ensemble[0].plot(label="prediction: Ensemble", ax=ax[i+1])
rmse_model =  res_ensemble[1]["RMSE"]
time_model =  res_ensemble[1]["time"]
ax[i+1].set_title("\n\n Ensemble: RMSE {:.0f}".format(rmse_model) + " - time {:.1f}sec".format(time_model))
ax[i+1].set_xlabel("")
ax[i+1].legend();


NameError: name 'models' is not defined